In [ ]:
# !pip install tensorflow==1.15.0

In [ ]:
# !pip install tensorflow-gpu==1.15.0

In [1]:
import sys
sys.path.append("..")
import gnn_utils

import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
import h5py
import time
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score
from collections import Counter
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

Using backend: pytorch
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Detected an old version of PyTorch. Suggest using torch>=1.5.0 for the best experience.
  return warnings.warn(message, category=category, stacklevel=1)
/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


In [2]:
tf.__version__

'1.15.0'

In [3]:
random_seed = [1111, 2222, 3333, 4444, 5555, 6666, 7777, 8888, 9999, 10000]

parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument("--dataname", default = "Quake_10x_Bladder", type = str)
parser.add_argument("--dataname", default = "data_-1.5c15", type = str)

parser.add_argument("--distribution", default = "ZINB")
parser.add_argument("--self_training", default = True)
parser.add_argument("--dims", default = [499, 256, 64, 32])
parser.add_argument("--highly_genes", default = 500)
parser.add_argument("--alpha", default = 0.001, type = float)
parser.add_argument("--gamma", default = 0.001, type = float)
parser.add_argument("--learning_rate", default = 0.0001, type = float)
parser.add_argument("--random_seed", default = random_seed)
parser.add_argument("--batch_size", default = 256, type = int)
parser.add_argument("--update_epoch", default = 10, type = int)
parser.add_argument("--pretrain_epoch", default = 1000, type = int)
parser.add_argument("--funetrain_epoch", default = 2000, type = int)
parser.add_argument("--t_alpha", default = 1.0)
parser.add_argument("--noise_sd", default = 1.5)
parser.add_argument("--error", default = 0.001, type = float)
parser.add_argument("--gpu_option", default = "0")

args = parser.parse_args("")

In [9]:
# category = "real_data"
for category in ["balanced_data", "imbalanced_data", #"real_data"
                ]:

    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)

    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")
        if X.shape[0] > 10000:
            continue

        X = np.ceil(X).astype(np.int)
        count_X = X
        print(X.shape, count_X.shape)
        orig_X = X.copy()
        adata = sc.AnnData(X)
        adata.obs['Group'] = Y
        adata = normalize(adata,
                          copy=True,
                          highly_genes=args.highly_genes,
                          size_factors=True,
                          normalize_input=True,
                          logtrans_input=True)
        X = adata.X.astype(np.float32)
        Y = np.array(adata.obs["Group"])

        high_variable = np.array(adata.var.highly_variable.index, dtype=np.int)
        count_X = count_X[:, high_variable]
        size_factor = np.array(adata.obs.size_factors).reshape(-1,
                                                               1).astype(np.float32)
        cluster_number = int(max(Y) - min(Y) + 1)
        args.dims[0]=X.shape[1]
        print(X.shape, count_X.shape)

        for run in range(3):
            start = time.time()
            seed = run
            np.random.seed(seed)
            tf.reset_default_graph()
            chencluster = autoencoder(args.dataname, args.distribution,
                                      args.self_training, args.dims, cluster_number,
                                      args.t_alpha, args.alpha, args.gamma,
                                      args.learning_rate, args.noise_sd)
            chencluster.pretrain(X, count_X, size_factor, args.batch_size,
                                 args.pretrain_epoch, args.gpu_option)

            chencluster.funetrain(X, count_X, size_factor, args.batch_size,
                                  args.funetrain_epoch, args.update_epoch, args.error, Y)

            kmeans_ARI = np.around(adjusted_rand_score(Y, chencluster.kmeans_pred), 5)
            kmeans_NMI = np.around(
                normalized_mutual_info_score(Y, chencluster.kmeans_pred), 5)

            elapsed = time.time() - start
            ARI = adjusted_rand_score(Y, chencluster.Y_pred)
            NMI = np.around(normalized_mutual_info_score(Y, chencluster.Y_pred), 5)
            ss = silhouette_score(X,chencluster.Y_pred)

            df.loc[df.shape[0]] = [dataset, ARI, NMI, ss, run, elapsed, chencluster.Y_pred]
            df.to_pickle(f"../output/pickle_results/{category}/{category}_sczi.pkl")

['data_1c8', 'data_-1c4', 'data_-1c8', 'data_0c4', 'data_0c8', 'data_0c16', 'data_1.5c4', 'data_1c4', 'data_1.5c8', 'data_1.5c16', 'data_-1c16', 'data_1c16']
>>>>dataset data_1c8
(2000, 2500) (2000, 2500)
(2000, 499) (2000, 499)
begin the pretraining
begin the funetraining
ARI 0.44063, NMI 0.50896
0 ARI 0.44063, NMI 0.50896
begin the pretraining
begin the funetraining
ARI 0.3879, NMI 0.43861
0 ARI 0.3879, NMI 0.43861
begin the pretraining
begin the funetraining
ARI 0.41902, NMI 0.49949
0 ARI 0.41956, NMI 0.50011
>>>>dataset data_-1c4
(1000, 2500) (1000, 2500)
(1000, 499) (1000, 499)
begin the pretraining
begin the funetraining
ARI 0.95403, NMI 0.92952
0 ARI 0.95403, NMI 0.92952
begin the pretraining
begin the funetraining
ARI 0.94286, NMI 0.91227
0 ARI 0.94286, NMI 0.91227
begin the pretraining
begin the funetraining
ARI 0.95153, NMI 0.92893
0 ARI 0.95153, NMI 0.92893
>>>>dataset data_-1c8
(2000, 2500) (2000, 2500)
(2000, 499) (2000, 499)
begin the pretraining
begin the funetraining
AR

1490 ARI 0.03873, NMI 0.09165
1500 ARI 0.03896, NMI 0.09294
1510 ARI 0.03842, NMI 0.09078
1520 ARI 0.03943, NMI 0.09367
1530 ARI 0.03883, NMI 0.0917
1540 ARI 0.03902, NMI 0.09211
1550 ARI 0.03951, NMI 0.09322
1560 ARI 0.0392, NMI 0.09247
1570 ARI 0.03922, NMI 0.09236
1580 ARI 0.03865, NMI 0.09147
1590 ARI 0.03927, NMI 0.09243
1600 ARI 0.03942, NMI 0.0928
1610 ARI 0.03945, NMI 0.09275
1620 ARI 0.04028, NMI 0.09373
1630 ARI 0.04025, NMI 0.0938
1640 ARI 0.04009, NMI 0.09314
1650 ARI 0.04137, NMI 0.09503
1660 ARI 0.04056, NMI 0.09303
1670 ARI 0.04089, NMI 0.09415
1680 ARI 0.04144, NMI 0.09477
1690 ARI 0.04161, NMI 0.09553
1700 ARI 0.04143, NMI 0.09492
1710 ARI 0.0424, NMI 0.0962
1720 ARI 0.04139, NMI 0.0945
1730 ARI 0.04194, NMI 0.0958
1740 ARI 0.04237, NMI 0.096
1750 ARI 0.04272, NMI 0.09699
1760 ARI 0.04224, NMI 0.09629
1770 ARI 0.0436, NMI 0.09808
1780 ARI 0.04271, NMI 0.09671
1790 ARI 0.04324, NMI 0.09772
1800 ARI 0.04292, NMI 0.09728
1810 ARI 0.04398, NMI 0.09891
1820 ARI 0.04328, NMI

230 ARI 0.05576, NMI 0.11267
240 ARI 0.05613, NMI 0.11269
250 ARI 0.05609, NMI 0.11285
260 ARI 0.05631, NMI 0.11324
270 ARI 0.0565, NMI 0.11343
280 ARI 0.05661, NMI 0.11393
290 ARI 0.05661, NMI 0.1139
300 ARI 0.05684, NMI 0.1143
310 ARI 0.05688, NMI 0.1143
320 ARI 0.0573, NMI 0.11476
330 ARI 0.05764, NMI 0.11535
340 ARI 0.05716, NMI 0.11476
350 ARI 0.05741, NMI 0.11508
360 ARI 0.05766, NMI 0.11578
370 ARI 0.05826, NMI 0.11622
380 ARI 0.05839, NMI 0.11623
390 ARI 0.05889, NMI 0.11714
400 ARI 0.0591, NMI 0.11701
410 ARI 0.0595, NMI 0.11749
420 ARI 0.06009, NMI 0.11839
430 ARI 0.05958, NMI 0.11777
440 ARI 0.05971, NMI 0.11797
450 ARI 0.05976, NMI 0.11808
460 ARI 0.06011, NMI 0.11839
470 ARI 0.06029, NMI 0.11889
480 ARI 0.05991, NMI 0.11855
490 ARI 0.06051, NMI 0.11979
500 ARI 0.05989, NMI 0.1187
510 ARI 0.06051, NMI 0.11984
520 ARI 0.06032, NMI 0.11952
530 ARI 0.06059, NMI 0.12026
540 ARI 0.06077, NMI 0.12074
550 ARI 0.06061, NMI 0.12023
560 ARI 0.06076, NMI 0.12039
570 ARI 0.06113, NMI 0

670 ARI 0.12657, NMI 0.21994
680 ARI 0.1275, NMI 0.22134
690 ARI 0.12723, NMI 0.22051
700 ARI 0.12772, NMI 0.22115
710 ARI 0.12778, NMI 0.22167
720 ARI 0.12741, NMI 0.22104
730 ARI 0.12802, NMI 0.22228
740 ARI 0.12847, NMI 0.22247
750 ARI 0.1297, NMI 0.2235
760 ARI 0.12978, NMI 0.22364
770 ARI 0.12991, NMI 0.22299
780 ARI 0.12968, NMI 0.22393
790 ARI 0.1301, NMI 0.2233
800 ARI 0.13107, NMI 0.22382
810 ARI 0.13211, NMI 0.22489
820 ARI 0.13072, NMI 0.22261
830 ARI 0.13316, NMI 0.22805
840 ARI 0.13231, NMI 0.22535
850 ARI 0.13218, NMI 0.22536
860 ARI 0.13334, NMI 0.22636
870 ARI 0.13369, NMI 0.22679
880 ARI 0.13442, NMI 0.22764
890 ARI 0.13351, NMI 0.22647
900 ARI 0.13432, NMI 0.22769
910 ARI 0.13597, NMI 0.22954
920 ARI 0.1365, NMI 0.22961
930 ARI 0.1356, NMI 0.2288
940 ARI 0.13588, NMI 0.22931
950 ARI 0.13503, NMI 0.22832
960 ARI 0.13589, NMI 0.22957
970 ARI 0.13734, NMI 0.23153
980 ARI 0.13692, NMI 0.23154
990 ARI 0.13699, NMI 0.23188
1000 ARI 0.13745, NMI 0.23177
1010 ARI 0.13914, NMI

1440 ARI 0.2319, NMI 0.32964
1450 ARI 0.23201, NMI 0.33102
1460 ARI 0.23406, NMI 0.33342
1470 ARI 0.23301, NMI 0.3295
1480 ARI 0.23984, NMI 0.33722
1490 ARI 0.24143, NMI 0.33809
1500 ARI 0.24221, NMI 0.33868
1510 ARI 0.23556, NMI 0.33307
1520 ARI 0.24248, NMI 0.34044
1530 ARI 0.24354, NMI 0.34223
1540 ARI 0.24397, NMI 0.34347
1550 ARI 0.24995, NMI 0.34591
1560 ARI 0.2489, NMI 0.34623
1570 ARI 0.25092, NMI 0.34772
1580 ARI 0.25212, NMI 0.34805
1590 ARI 0.2546, NMI 0.35185
1600 ARI 0.24757, NMI 0.34669
1610 ARI 0.25394, NMI 0.35074
1620 ARI 0.25451, NMI 0.35208
1630 ARI 0.25596, NMI 0.3541
1640 ARI 0.25711, NMI 0.35473
1650 ARI 0.2602, NMI 0.35809
1660 ARI 0.26107, NMI 0.35844
1670 ARI 0.26113, NMI 0.35842
1680 ARI 0.26212, NMI 0.36011
1690 ARI 0.26301, NMI 0.36115
1700 ARI 0.26331, NMI 0.36158
1710 ARI 0.26463, NMI 0.36362
1720 ARI 0.26511, NMI 0.3639
1730 ARI 0.26672, NMI 0.36573
1740 ARI 0.26819, NMI 0.36666
1750 ARI 0.26996, NMI 0.36874
1760 ARI 0.27263, NMI 0.37073
1770 ARI 0.2718, 

...

70 ARI 0.52651, NMI 0.55245
80 ARI 0.52883, NMI 0.55557
90 ARI 0.53367, NMI 0.559
100 ARI 0.53638, NMI 0.5613
110 ARI 0.53939, NMI 0.56335
120 ARI 0.54419, NMI 0.56801
130 ARI 0.5448, NMI 0.56853
begin the pretraining
begin the funetraining
ARI 0.39329, NMI 0.41203
0 ARI 0.39156, NMI 0.41099
10 ARI 0.38268, NMI 0.41219
20 ARI 0.3771, NMI 0.41271
30 ARI 0.38008, NMI 0.41623
40 ARI 0.37883, NMI 0.41587
50 ARI 0.37923, NMI 0.41789
60 ARI 0.38064, NMI 0.42029
70 ARI 0.37949, NMI 0.42126
80 ARI 0.3811, NMI 0.42361
90 ARI 0.38093, NMI 0.42503
100 ARI 0.38124, NMI 0.4251
110 ARI 0.38149, NMI 0.42664
120 ARI 0.38253, NMI 0.42857
130 ARI 0.38416, NMI 0.43102
140 ARI 0.38414, NMI 0.43209
150 ARI 0.38574, NMI 0.43311
160 ARI 0.38479, NMI 0.43377
170 ARI 0.3857, NMI 0.43489
180 ARI 0.38602, NMI 0.43584
190 ARI 0.38696, NMI 0.43799
200 ARI 0.38641, NMI 0.43693
210 ARI 0.3864, NMI 0.43837
220 ARI 0.38742, NMI 0.43952
230 ARI 0.38774, NMI 0.44159
240 ARI 0.38855, NMI 0.4415
250 ARI 0.38943, NMI 0.445

640 ARI 0.45261, NMI 0.51774
650 ARI 0.44921, NMI 0.51705
660 ARI 0.45264, NMI 0.51851
670 ARI 0.45082, NMI 0.51963
680 ARI 0.45224, NMI 0.51842
690 ARI 0.44972, NMI 0.51789
700 ARI 0.45272, NMI 0.51854
710 ARI 0.44944, NMI 0.51681
720 ARI 0.45184, NMI 0.51828
730 ARI 0.44988, NMI 0.51703
740 ARI 0.44902, NMI 0.51453
750 ARI 0.44972, NMI 0.51576
760 ARI 0.45031, NMI 0.51551
770 ARI 0.44933, NMI 0.51532
780 ARI 0.44839, NMI 0.51423
790 ARI 0.45021, NMI 0.5162
800 ARI 0.45006, NMI 0.51626
810 ARI 0.45124, NMI 0.51698
820 ARI 0.44869, NMI 0.51563
830 ARI 0.44864, NMI 0.51475
840 ARI 0.44896, NMI 0.51563
850 ARI 0.44938, NMI 0.51493
860 ARI 0.44822, NMI 0.51457
870 ARI 0.44769, NMI 0.51336
880 ARI 0.4478, NMI 0.51466
890 ARI 0.447, NMI 0.51196
900 ARI 0.44809, NMI 0.5145
910 ARI 0.44681, NMI 0.51164
920 ARI 0.4482, NMI 0.51449
930 ARI 0.44765, NMI 0.51213
940 ARI 0.44789, NMI 0.51317
950 ARI 0.4475, NMI 0.51219
960 ARI 0.4478, NMI 0.51382
970 ARI 0.44843, NMI 0.51396
980 ARI 0.44849, NMI 0

1410 ARI 0.41082, NMI 0.42962
1420 ARI 0.41579, NMI 0.43261
1430 ARI 0.41085, NMI 0.43046
1440 ARI 0.41421, NMI 0.43115
1450 ARI 0.41245, NMI 0.43046
1460 ARI 0.41454, NMI 0.43036
1470 ARI 0.41345, NMI 0.43195
1480 ARI 0.41583, NMI 0.43169
1490 ARI 0.4136, NMI 0.43151
1500 ARI 0.41503, NMI 0.43096
1510 ARI 0.41459, NMI 0.43307
1520 ARI 0.41342, NMI 0.42955
1530 ARI 0.41466, NMI 0.43332
1540 ARI 0.41609, NMI 0.43149
1550 ARI 0.41536, NMI 0.43318
1560 ARI 0.41314, NMI 0.43027
1570 ARI 0.41541, NMI 0.43336
1580 ARI 0.41677, NMI 0.43389
1590 ARI 0.41468, NMI 0.43256
1600 ARI 0.41526, NMI 0.43188
1610 ARI 0.41623, NMI 0.43336
1620 ARI 0.41412, NMI 0.43171
1630 ARI 0.41454, NMI 0.43233
1640 ARI 0.41588, NMI 0.43409
1650 ARI 0.41602, NMI 0.4334
1660 ARI 0.41384, NMI 0.43193
1670 ARI 0.41692, NMI 0.43407
1680 ARI 0.41523, NMI 0.43318
1690 ARI 0.4165, NMI 0.43456
1700 ARI 0.41644, NMI 0.43396
1710 ARI 0.41782, NMI 0.43516
1720 ARI 0.41558, NMI 0.43338
1730 ARI 0.41603, NMI 0.43364
1740 ARI 0.41

1660 ARI 0.15489, NMI 0.19898
1670 ARI 0.15807, NMI 0.20177
1680 ARI 0.15418, NMI 0.19918
1690 ARI 0.15647, NMI 0.20133
1700 ARI 0.15516, NMI 0.19975
1710 ARI 0.15787, NMI 0.2021
1720 ARI 0.15601, NMI 0.20112
1730 ARI 0.15583, NMI 0.2016
1740 ARI 0.15637, NMI 0.20096
1750 ARI 0.1564, NMI 0.20067
1760 ARI 0.15636, NMI 0.20173
1770 ARI 0.15832, NMI 0.20468
1780 ARI 0.15767, NMI 0.20275
1790 ARI 0.15835, NMI 0.20337
1800 ARI 0.15738, NMI 0.20314
1810 ARI 0.15956, NMI 0.2052
1820 ARI 0.16057, NMI 0.2061
1830 ARI 0.16008, NMI 0.206
1840 ARI 0.15882, NMI 0.20492
1850 ARI 0.15942, NMI 0.20488
1860 ARI 0.15898, NMI 0.20514
1870 ARI 0.16413, NMI 0.20851
1880 ARI 0.16028, NMI 0.20584
1890 ARI 0.15987, NMI 0.20584
1900 ARI 0.15932, NMI 0.20493
1910 ARI 0.166, NMI 0.20964
1920 ARI 0.16236, NMI 0.20697
1930 ARI 0.16316, NMI 0.20813
1940 ARI 0.15986, NMI 0.2059
1950 ARI 0.16695, NMI 0.21159
1960 ARI 0.16683, NMI 0.21059
1970 ARI 0.1668, NMI 0.21096
1980 ARI 0.16146, NMI 0.20779
1990 ARI 0.16712, NMI

90 ARI 0.24801, NMI 0.31941
100 ARI 0.24784, NMI 0.31921
110 ARI 0.24866, NMI 0.31989
120 ARI 0.24921, NMI 0.32027
130 ARI 0.2503, NMI 0.32146
140 ARI 0.24991, NMI 0.32095
150 ARI 0.25007, NMI 0.32147
160 ARI 0.25108, NMI 0.32316
170 ARI 0.2503, NMI 0.32279
180 ARI 0.25055, NMI 0.32301
190 ARI 0.25189, NMI 0.32472
200 ARI 0.25333, NMI 0.32559
210 ARI 0.25445, NMI 0.32717
220 ARI 0.25426, NMI 0.32743
230 ARI 0.25688, NMI 0.33037
240 ARI 0.25737, NMI 0.33049
250 ARI 0.25658, NMI 0.33032
260 ARI 0.25737, NMI 0.33119
270 ARI 0.25847, NMI 0.3324
280 ARI 0.261, NMI 0.3342
290 ARI 0.26039, NMI 0.33433
300 ARI 0.26092, NMI 0.33457
310 ARI 0.26105, NMI 0.33501
320 ARI 0.26087, NMI 0.33489
330 ARI 0.26088, NMI 0.33511
340 ARI 0.26179, NMI 0.33616
350 ARI 0.26178, NMI 0.33584
360 ARI 0.26312, NMI 0.33796
370 ARI 0.26352, NMI 0.33831
380 ARI 0.26398, NMI 0.33919
390 ARI 0.26383, NMI 0.33886
400 ARI 0.26631, NMI 0.34084
410 ARI 0.26644, NMI 0.34111
420 ARI 0.27065, NMI 0.34499
430 ARI 0.26858, NMI 

In [8]:
df.head()

,dataset,ARI,NMI,sil,run,time,pred
0,Quake_Smart-seq2_Trachea,0.826206,0.73851,0.159376,0,72.852339,"[1, 3, 0, 3, 3, 1, 0, 3, 2, 1, 1, 0, 1, 1, 1, ..."
1,Quake_Smart-seq2_Trachea,0.845888,0.73748,0.170133,1,74.360769,"[0, 3, 2, 3, 3, 1, 2, 3, 0, 1, 1, 2, 1, 1, 1, ..."
2,Quake_Smart-seq2_Trachea,0.815244,0.71714,0.192821,2,74.582662,"[3, 2, 1, 2, 2, 3, 1, 2, 0, 3, 3, 1, 3, 3, 3, ..."
3,Quake_Smart-seq2_Diaphragm,0.961073,0.93895,0.064950,0,49.484904,"[1, 0, 0, 3, 3, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Quake_Smart-seq2_Diaphragm,0.956542,0.93237,0.066473,1,49.456714,"[2, 0, 0, 3, 0, 1, 4, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [6]:
df.groupby("dataset").mean()

,ARI,NMI,sil,time
dataset,,,,
10X_PBMC,0.584565,0.706333,0.064692,460.356967
10X_PBMC_select_2100,0.597950,0.706143,0.026049,100.385072
Adam,0.863456,0.857260,0.021948,174.062563
Muraro,0.724627,0.777293,0.047724,142.597028
Quake_10x_Bladder,0.983096,0.966417,0.119126,123.532811
Quake_10x_Limb_Muscle,0.961056,0.933257,0.026476,249.786629
Quake_10x_Spleen,0.909535,0.843680,0.209160,466.575191
Quake_Smart-seq2_Diaphragm,0.959670,0.935230,0.063294,49.739813
Quake_Smart-seq2_Limb_Muscle,0.973676,0.951237,0.078811,54.004468
